In [1]:
import pandas as pd
import numpy as np
import multiprocessing
from bs4 import BeautifulSoup as bs
from selectolax.parser import HTMLParser
import re
import pymorphy2
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import string

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import keyboard
from time import time  # To time our operations
from collections import defaultdict, Counter  # For word frequency

# import logging  # Setting up the loggings to monitor gensim
# logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

import warnings
warnings.filterwarnings('ignore')

2023-03-27 16:57:32.336185: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/adwiz/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/adwiz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# Data loading and preparation
data = pd.read_json('../datasets/dataset.json')
mapping = {False: 0, True: 1}
data.replace({'hasBadWords': mapping}, inplace=True)
# data.hasBadWords = data.hasBadWords.apply(lambda x: 1 if x == True else 0)
data.rename(columns={"hasBadWords": "labels"}, inplace=True)
data.drop(['violation'], axis=1, inplace=True)
data.shape

(86439, 2)

In [4]:
data.head()

,text,labels
0,My Favorite Slut,0
1,girlfriends sit on each other's faces with the...,0
2,bound beauty kisses her girlfriend,0
3,MORGAN - Anytime - Nail Painting On The Slave'...,0
4,TRANSGENDER COACHING (wmv) PART 1,0


# Preprocessing function

In [5]:
def preprocess(text, stop_words, punctuation_marks): #, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = token #morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return ' '.join(preprocessed_text)

punctuation_marks = ['!', ',', ';', "'", '(', ')', ':', '-', '--', '', '?', '@', '....', '~',
                     '.', '..', '...', '<', '>', '=', '\'\'', '//', '»', '|', '’', '`', '+'
                     '\"\"', '</', '&', '/', '#', '\'', '*', '``', '%', '[', ']', '{', '}']

stop_words = stopwords.words('english') + ['14000kbps', "n't", "'s", "'m", 'mp4', 'error', '404']
morph = pymorphy2.MorphAnalyzer(lang='uk')

In [6]:
def normalize(s):
    """
    Given a text, cleans and normalizes it. Feel free to add your own stuff.
    """
    s = s.lower()
    # Replace ips
    s = re.sub(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', ' _ip_ ', s)
    # Isolate punctuation
    s = re.sub(r'([.\(\)\!\?\-\\\/\,])', r' \1 ', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Replace numbers and symbols with language
    s = s.replace('&', ' and ')
    s = s.replace('@', ' at ')
    s = s.replace('0', ' zero ')
    s = s.replace('1', ' one ')
    s = s.replace('2', ' two ')
    s = s.replace('3', ' three ')
    s = s.replace('4', ' four ')
    s = s.replace('5', ' five ')
    s = s.replace('6', ' six ')
    s = s.replace('7', ' seven ')
    s = s.replace('8', ' eight ')
    s = s.replace('9', ' nine ')
    return s

In [ ]:
data[:1000].apply(lambda row: str(row.text), axis=1)

In [7]:
# Dropping the rows with "Sci"
# identify partial string
# discard = ["<dev"]
  
# # drop rows that contain the partial string "dev"
# data[~data.text.str.contains('|'.join(discard))]

data = data[data['text'].str.contains('<div')==False]

In [8]:
data

,text,labels
0,My Favorite Slut,0
1,girlfriends sit on each other's faces with the...,0
2,bound beauty kisses her girlfriend,0
3,MORGAN - Anytime - Nail Painting On The Slave'...,0
4,TRANSGENDER COACHING (wmv) PART 1,0
...,...,...
83594,"ebony,hotwife,wife,swinger,cuckold,bigass",0
83595,"ssbhm, bhm, ffa, female fat admire, fat admire...",0
83596,"Feet in heels, sexy shoes, high heels, high he...",0
83597,"foot fetish, breeding, kinky, fetish porn, bon...",0


In [ ]:
# data[:100].apply(lambda row: bs(row['text']).get_text().replace('\n',' '),axis=1)

In [ ]:
data[:100].apply(lambda row: preprocess(row.text, punctuation_marks, stop_words), axis=1) #, morph), axis=1)

In [ ]:
# data['text_preprocessed'] = data.apply(lambda row: bs(row.text).get_text().replace('\n',' '),axis=1)
# data['text_preprocessed'] = data.apply(lambda row: HTMLParser(row.text).body.text(separator=' ').replace('\n',' '),axis=1)

In [9]:
data['text_preprocessed'] = data.apply(lambda row: preprocess(row.text, punctuation_marks, stop_words), axis=1)

In [10]:
data = data[['text_preprocessed', 'labels', 'text']] # columns reorder

In [11]:
data.head()

,text_preprocessed,labels,text
0,favorite slut,0,My Favorite Slut
1,girlfriends sit faces asses,0,girlfriends sit on each other's faces with the...
2,bound beauty kisses girlfriend,0,bound beauty kisses her girlfriend
3,morgan anytime nail painting slave face,0,MORGAN - Anytime - Nail Painting On The Slave'...
4,transgender coaching wmv part 1,0,TRANSGENDER COACHING (wmv) PART 1


# Tensorflow

In [ ]:
# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count  

In [ ]:
counter = counter_word(data.text_preprocessed)  

In [ ]:
data.text_preprocessed

In [ ]:
num_unique_words = len(counter)

In [ ]:
num_unique_words

In [ ]:
counter

In [ ]:
counter.most_common(5)

In [ ]:
from wordcloud import *
word_freq = [i for i in counter.most_common(50)]
wd = WordCloud(background_color='white')
wd.generate_from_frequencies(dict(word_freq))
plt.figure()
plt.imshow(wd, interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
train_size = int(data.shape[0] * .8)

train = data[:train_size]
val = data[train_size:]

# Split train and test

X_train = train.text_preprocessed.to_numpy()
y_train = train.labels.to_numpy()

X_val = val.text_preprocessed.to_numpy()
y_val = val.labels.to_numpy()

In [ ]:
train.text_preprocessed.to_numpy()

In [ ]:
X_train,  y_train

In [ ]:
X_train.shape, X_val.shape

# Tokenize

In [ ]:
# vectorize a text corpus by turning each text into sentence of integers

tokenizer2 = Tokenizer(num_words=num_unique_words)
tokenizer2.fit_on_texts(X_train)

In [ ]:
import pickle

# saving
with open('../models/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer2, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('../models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
# each word have a unique index
word_index = tokenizer.word_index
word_index

In [ ]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_val_sequences = tokenizer.texts_to_sequences(X_val)

In [ ]:
print(X_train[1])
print(X_train_sequences[1])

In [ ]:
# pad the sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

# max words in a sequence
max_length = 20

X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length, padding="post", truncating="post")
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_length, padding="post", truncating="post")

X_train_padded.shape, X_val_padded.shape


In [ ]:
X_train_padded[10]

In [ ]:
print(X_train[10])
print(X_train_sequences[10])
print(X_train_padded[10])

In [ ]:
# Check reversing the indices

# flip (key, value)
reverse_word_index = dict([(idx, word) for (word, idx) in word_index.items()])

In [ ]:
reverse_word_index

In [ ]:
def decode(sequence):
    return " ".join([reverse_word_index.get(idx, "?") for idx in sequence])

decoded_text = decode(X_train_sequences[10])

print(X_train_sequences[10])
print(decoded_text)

In [ ]:
# Create LSTM model
from tensorflow.keras import layers

# Word embeddings give us a way to use an efficient, dense representation in which similar words have
# a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a 
# dense vector of floating point values (the length of the vector is a parameter you specify)

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))

# The layer will take as input as integer matrix of size (batcg, input_length)
# and the largest integer (i.e. word index) in the input should be no larger than num_words (vocabulary size).
# Now model.output_shape is (None, input_length, 32), where `None` is the batch dimension.

model.add(layers.LSTM(64, dropout=.1))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optimizer = keras.optimizers.Adam(learning_rate=.001)
metrics = ['accuracy']

model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

In [ ]:
model.fit(X_train_padded, y_train, epochs=15, validation_data=(X_val_padded, y_val), verbose=2)

In [ ]:
# model.get_weights()

In [ ]:
history_df = pd.DataFrame(model.history.history)
history_df.head()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history_df.loss);

In [ ]:
# predictions = model.predict(X_train_padded)
# predictions = [1 if p > .5 else 0 for p in predictions]

In [ ]:
# print(X_train[60:70])
# print(y_train[60:70])
# print(predictions[60:70])

In [ ]:
# train['predictions'] = predictions
# train = train[['text_preprocessed', 'labels', 'predictions', 'text']] # columns reorder

In [ ]:
# train.head()

In [ ]:
# train.to_csv('../datasets/lw2wtf.csv')

In [ ]:
model_new = keras.models.load_model('../models/modelSequential.h5')

In [ ]:
predictions = model_new.predict(X_train_padded)
predictions = [1 if p > .5 else 0 for p in predictions]

# Check sentence

In [ ]:
raw_text = "<p><em><strong>Cadence Lux, Tina Lee Comet &amp; Nate Liquor</strong></em></p>\r\n<p>Tina has a hard time finding hot ladies to tie up and fuck. So she hires an older, down on his luck guy, Nate, to catfish for her kinky needs on tindr to lure in a hottie to tie up. Isn't it just Tina's luck that on her first try that Cadence is a lezbo slut who loves being tied up.</p>\r\n<div class=\"col-sm-12 marginTop10\"><strong>Additional Formats</strong><br /><a href=\"https://www.clips4sale.com/studio/61965/26849831\">HD</a><br /><a href=\"https://www.clips4sale.com/studio/61965/26849837\">MOBILE</a> <br /><br /><strong>You'll also love</strong> <br /><a href=\"https://www.clips4sale.com/studio/61965/25840157\"> <img src=\"https://imagecdn.clips4sale.com/accounts99/61965/clip_images/WifeHusbandAndCoffeeShopPussy.gif\" alt=\"\" width=\"200\" height=\"113\" /> <br />The Wife, The Husband And The Coffee Shop Pussy</a><br /><a href=\"https://www.clips4sale.com/studio/61965/25550753\"> <img src=\"https://imagecdn.clips4sale.com/accounts99/61965/clip_images/FallingForHerAndHerTrap.gif\" alt=\"\" width=\"200\" height=\"113\" /> <br />Falling For Her And Her Trap</a></div>"
test_text_preprocessed = bs(raw_text).get_text().replace('\n',' ')
test_text_preprocessed = preprocess(test_text_preprocessed, punctuation_marks, stop_words)
test_text_np = np.array([test_text_preprocessed])
test_sequences = tokenizer.texts_to_sequences(test_text_np)

In [ ]:
print(test_text_preprocessed)
print(test_sequences[0])

In [ ]:
test_sequences_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post", truncating="post")

In [ ]:
%%time
predictions = model_new.predict(test_sequences_padded)
predictions = [1 if p > .5 else 0 for p in predictions]

In [ ]:
predictions

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# max words in a sequence
max_length = 20

from time import time  # To time our operations
from collections import defaultdict, Counter  # For word frequency

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

import warnings
warnings.filterwarnings('ignore')

# max words in a sequence
max_length = 20

from time import time  # To time our operations
from collections import defaultdict, Counter  # For word frequency

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

import warnings
warnings.filterwarnings('ignore')

def preprocess(text, stop_words, punctuation_marks): #, morph):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = token #morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return ' '.join(preprocessed_text)

punctuation_marks = ['!', ',', ';', "'", '(', ')', ':', '-', '--', '', '?', '@', '....', '~',
                     '.', '..', '...', '<', '>', '=', '==', '\'\'', '//', '»', '|', '’', '`', '+',
                     '\"\"', '</', '&', '/', '#', '\'', '*', '``', '%', '[', ']', '{', '}', '$']


stop_words = stopwords.words('english') + ['14000kbps', 'https', "'s", "'m", 'http', 'mp4', 'error', '404']

# loading
with open('../models/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
    
model = keras.models.load_model('../models/modelSequential.h5')

raw_text = '<p>Socks. Such a simple, common and necessary item. I love wearing socks. Frilly socks, fuzzy socks, reinforced toe and heel socks, sweat socks, lace socks, statement socks! I love finding cute colors to match and contrast my shoes and clothes. I actually sold many pairs of socks during college to cover my club visits. Lucky boys. Also, for animals that do not know how to shut their mouths. \"Put a sock in it\" literally does work. It is kind of weird how much I enjoy hearing the slurping and sucking sounds coming from the floor beneath me. \"Yes, I want that sock sucked cleaned foot mutt. Keep that mouth busy for me like a good boy.\"</p>'


def predict(sequences):
    sequences_padded = pad_sequences(sequences, maxlen=20, padding="post", truncating="post")
    predictions = model.predict(sequences_padded)
    predictions = [1 if p > .5 else 0 for p in predictions]
    return predictions

# Data loading and preparation
data = pd.read_json('../datasets/neil_ProducerClipSite_rand.json')
mapping = {False: 0, True: 1}
# data.replace({'hasBadWords': mapping}, inplace=True)
# data.hasBadWords = data.hasBadWords.apply(lambda x: 1 if x == True else 0)
# data.rename(columns={"hasBadWords": "labels"}, inplace=True)
data.drop(['SiteID', 'Title', 'Description', 'Keywords', 'Bottom'], axis=1, inplace=True)
data.head()



In [ ]:
data.Top

In [ ]:
%%time
data['text_preprocessed'] = data.apply(lambda row: bs(row.Top, 'lxml').get_text().replace('\n',' '), axis=1)
data['text_preprocessed'] = data.apply(lambda row: preprocess(row.text_preprocessed, punctuation_marks, stop_words), axis=1)

X_val = data.text_preprocessed.to_numpy()
X_val_sequences = tokenizer.texts_to_sequences(X_val)
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_length, padding="post", truncating="post")

predictions = model.predict(X_val_padded)
predictions = [1 if p > .5 else 0 for p in predictions]


In [ ]:
data['predictions'] = predictions

In [ ]:
data.head()

In [ ]:
data = data[['Top', 'predictions']]

In [ ]:
data.to_csv('../datasets/last_one.csv')

In [ ]:
data.to_json('../datasets/last_one.json')